In [ ]:
import numpy as np
import pymol
import os
import shutil
import lmdb
import pickle as pkl
import pymol

from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

In [ ]:
#Preprocess the collaborator sequenced antibody.
seq_path = 'seq from collaborator/'
file_list = os.listdir(seq_path)
input = open(seq_path + file_list[0]).readlines()
seq_dict = {}
for file_i in file_list:
    input = open(seq_path + file_i).readlines()
    contig_id = []
    chain_id = []
    seq = []
    seq_id = []
    for i in input[1:]:
        row = i.split(',')
        seq_id.append(row[0])
        contig_id.append(row[5].split('_')[0].split('clonotype')[1])
        if((row[14] == 'H') | (row[14] == 'M') | (row[14] == 'n')):
            chain_id.append('H')
        elif((row[14] == 'K') | (row[14] == 'L')):
            chain_id.append('L')
        else:
            print(row)
        seq.append(row[16])
    seq_id = np.array(seq_id)
    contig_id = np.array(contig_id)
    chain_id = np.array(chain_id)
    contig_unique = np.unique(np.array(contig_id))
    for contig_i in contig_unique:
        if(('H' in chain_id[np.where(contig_id == contig_i)]) & ('L' in chain_id[np.where(contig_id == contig_i)])):
            heavy_seq_index = np.where((contig_id == contig_i) & (chain_id == 'H'))[0][0]
            light_seq_index = np.where((contig_id == contig_i) & (chain_id == 'L'))[0][0]
            seq_dict[seq_id[heavy_seq_index] + '_' + seq_id[light_seq_index]] = {'H': seq[heavy_seq_index], 'L': seq[light_seq_index]}

In [3]:
#the sequences of the first batch antibodies (first 25)
seq_dict_batch1_cloned = {
            '1_2':   {'H': 'MDWTWRILFLVAAATGAHSQVQLVQSGAEVKKPGASVKVSCKASGYTFTDYYIHWVRQAPGQGLEWMGWINPISGGTNYAQKFQGRVTMTRDTSVTTFYMELSWLTSDDSAVYYCARCPFFYSETSGYFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MAWALLLLTLLTQGTGSWAQSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWFQQHPGKAPKLMIYEVSTRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSNTYVFGTGTQVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASS'},  
            '7_8':   {'H': 'MELGLSWVFLVAILEGVQCEVRLVESGGGLVQPGGSLRLSCAASGFTFSSYDMHWVRQATGKGLEWVSTIGTAGDTYYLGSVKGRFTFSRENAKNSLYLQMNSLRAGDTAVYYCARAKYYDSSGYYHYTPYYFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLPGTRCDIQMTQSPSSLSASVGDRVTITCRASLGIGNSLAWYQQKPGKAPKVLLYAASRLESGVPSRFSGSGSGTDYTLTISSLQPEDFATYYCQQYYSTPEVTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '11_12': {'H': 'MEFGLSWVFLVALLRGIQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSRYGMHWVRQAPGKGLEWVAVISFDGRNKYYADSVKGRFTISRDNSKTTLYLQMNSLRAEDTAVYHCAKDGLAVSDYLDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLRGARCDIQMTQSPSSLSASVGDRVTITCRASQSITNYLNWYQQKPGKAPELLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYSTPGTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '13_14': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVAQPGRSLRLSCAASGFTFSNYAMHWVRQAPGKGLEWVAVISFDGSDKYYVDSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKSGGLYCNGGNCYYGYYFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MDMRVPAQLLGLLLLWLPGAKCDIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYEASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNTYSPYTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '15_16': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYMQMNTLRAEDTAVYYCAKVVGAYCGGDCLTGYFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKLLIYDASNLETGVPSRFSGSASGTDFTFTISSLQPEDIATYYCQQYDNLPLTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '17_18': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSSHVMHWVRQTPGKGLEWVAVISYDGSSKYYADSVKGRFTISRDNAKNTLYLQMNSLKTEDTAVYYCARERVSSGWYLDPFDIWGQGTMVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLRGARCDIQMTQSPSSLSASVGDRVTITCRASQSISNYLNWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYTTLSITFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '19_20': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSNYAMHWVRQAPGKGLEWVAVISYDGSNKYYVDSVKGRFTISRDNSKSTLYLQINSLRAEDTAVYYCARDRKPSYDSWSGYTHYHYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MAWALLLLTLLTQDTGSWAQSALTQPASVSGSRRQSITISCTGTSSDVGSYNLVSWFQHHPGKAPNLVIYEDNKRPSGVSNRFSGSKSGHTASLTISGLQAEDEADYYCCSYAGSGTWVFGGGTKLTVLSQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '25_26': {'H': 'MKHLWFFLLLVAAPRWVLSQVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIFYSGSTNYNPSLRSRVTISVDTPKNQFSLRLRSVTAADTAVYYCARDSMDTTTWAPTAFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKLLIYDASNLETGVPSRFNGSGSGTDFTFTISSLQPEDIATYYCQQYDNLPLTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '31_32': {'H': 'MDWTWRILFLVAAATGAHSQVQLVQSGAEVKKPGASVKVSCKASGYFFTGFYIHWVRQAPGQGLEWMGWISPISGGANSAQTFQDRVTMTRDTSITTAYMELSRLRSDDTAVYYCARAPYYDSSASLDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQASQDISNSLNWYHQKPGKAPRLLIYDASNLKTGVPSRFSGSGSGTDFTFTISSLQPEDIGTFYCQQYDNLPPALTFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '39_40': {'H': 'MELGLSWVFLVAILEGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTFSRYDMHWVRQATGKGLEWVSAIGTSGDTYYPGSVRGRFTISRENAKNSLYLQMNSLRAGDTAVYYCARVNYDSGGYGIREYWFFDLWGRGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLRGSRCDIQMTPSPSSLSASVGDRVTITCRASQSISSFLNWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYFCQQSYSTPPWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '43_44': {'H': 'MDFGLSWLFLVAILKGVQCEVQLLESGGGLVQPGGSLRLSCSASGFTFGTYAMSWVRQAPGKGLECVSTIDDIYGSGGRTFYAGSVHGRFTISRDNSKNTLYLQMNSLRAEDTAIYYCARDKYHYDSGGYYRLAGLDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLPGARCAIQLTQSPSSLSASVGDRVTITCRASQGISSYLAWYQQKPGKAPKLLIFGASTLQSGAPSRFRGSGSGTDFTLAISNLQPEDFATYYCQQTDSYPRTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '51_52': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVASISYDGSEYYAESVKGRFTISRDNSKSTLHLQMKSLRAEDTAVYYCAKNGGPYCSGGGCYGSYFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLRGARCDIQMTQSPPSLSASVGDRVTITCRASQSISSYLNWYQQKPGNAPKLLIFAASSLETGVPSRFSGSGSGTDFTLTINSLQPEDFATYYCQQSSSAPLTFGPGAEVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '86_87': {'H': 'MGSTAILALLLAVLQGVCAEVQLVQSGAEVKKPGESLKISCKGSGYSFISNWIGWVRQMPGKGLEWMGSIYPGDSDTRYSPSFQGQVTISADKSISTAYLQWSSLKASDTAIYYCARLESDWYFDLWGRGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MAWTPLWLTLLTLCIGSVVSSELTQDPAVSVALGQTVRITCQGDSLRSHYASWYQQKPGQAPVVVIYGKDNRPSGIPDRFSGSSSGNTASLTITGAQAEDEADYYCNSRDSSGNHPFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '88_89': {'H': 'MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSLIYSGGSTYYADSVKGRFTISRDYSKNTLYLQMNSLRAEDTATYYCARERPRGAGEYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQASQDINIYLNWYQQKPGKAPKLLIYDASNLETGVPSRFSGSGSGTDFTFTISSLQPEDIATYYCHQHDNLPRTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '92_93': {'H': 'MDWTWRVFCLLAVAPGAHSQVQLVQSGAEVQKPGASVKVSCKASGHTFTSYYIHWVRQAPGQGLEWMGILNTSGGSTTYAQKFQGRVTMTRDTSTSTVYMDLSSLRSEDTAVYYCASSSWDDAFDIWGQGTMVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MAWTPLFLFLLTCCPGSNSQTVVTQEPSLTVSPGGTVTLTCASSTGAVTSGYYPNWFQQKPGQAPRALIYSTRNKHSWTPARFSGSLLGGRAALTLSGVQPEDEAEYYCLLYYGGPWVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'},
            '94_95': {'H': 'MDWTWRVFCLLAVAPGAHSQVQLVQSGAEVKKPGASVKVSCKASGYTFMNYYMHWVRQAPGQGLEWMGMINPSGGSATYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAREERGCSTTSCYDDAFDIWGQGTMVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                      'L': 'MDMRVPAQLLGLLLLWFPGARCDIQMTQSPSAMSASVGDRVTITCRASQGISNYLAWFQQKPGKVPKRLIYAASSLQSGVPSRFSGSGSGTEFTLTISSLQPEDFATYYCLQHNSYPWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '96_97': {'H': 'MDWTWRVFCLLAVAPGAHSEVQLVQSGAEVKKPGASVKVSCKASGYTFITYYIHWMRQAPGQGLEWMGLINPSGGSTNFAQNFQGRVTMTRDTSTSTVHMELTSLRSEDTAVYYCARGDSGYSSSWCDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MAWTPLWLTLLTLCIGSVVSSEVTQDPAVSVALGQTVRITCQGDSLRSYSASWYQQRPGQAPVLVIYAKDNRPSGIPVRFSGSSSGTTASLTITGAQAEDEADYYCSSRDSSDTVLFGGGTKLTVLSQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '100_101': {'H': 'MEWTWRFLFVVAAATGVQSQVQLVQSGAEVKKPGSSVKVSCKASGGTFSYYAINWVRQAPGQGLECMGRIIPFLGIANYTQRFQGRVTITADKSTSTAYMELRSLRSEDTAVYFCAREGPYYYDSSGYSKSDSDGMDVWGQGTTVTVSAASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                        'L': 'MDMRVPAQLLGLLLLWLPGARCAIQLTQSPSSLSASVGDRVTITCRASQGISSYLAWYQQKPGKAPKLLLYASSTLPSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQLNSYPPTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '104_105': {'H': 'MELGLSWIFLVAILKGVQCEVQLVESGGGVVRPGGSLRLSCAASGFTYDTYGMSWVRQAPGKGLEWVSGINWNGGRSGYADSVKGRFIIFRDNAKNSLYLQMNSLRVEDTALYYCARASVGYCASSRCSNWFDTWGQGTLVTVSSASTKGPSVFPLAPCSRSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                        'L': 'MTCSPLLLTLLIHCTGSWAQSVLTQPPSLSAAPGQKVTISCSGSSSNIGNNYVSWYQQLPGAAPKLLIYDNNKRPSGISDRFSGSMSGTSATLGITGLQTGDEGDYYCGTWDSSLSLVVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'},
            '106_107': {'H': 'MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGGSLRLSCVASGITVSANYMSWVRQAPGKGLEWVSVIYSGGSTFYADSVKGRFTISRDNSKNTLYLQMNNLRADDTAVYSCARDFRGATAFDIWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                        'L': 'MAWALLFLTLLTQGTGSWAQSALTQPASVSGSPGQSITISCTATSSDVDDYNYVSWYQQHPGKAPKLLIYDVNNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSSTGVFGSGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASS'},
            '108_109': {'H': 'MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSVIYSGGTTYYADSVKGRFTLSRDNSKNTLYLQMNSLRAEDTAVYYCARGSYDSSGLVMSGAFDIWGQGTMVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                        'L': 'MAWALLLLTLLTQGTGSWAQSALTQPPSASGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYEVSKRPSGVPDRFSGSKSGNTASLTVSGLQAEDEADYYCSSYAGSNNWVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'},
            '110_111': {'H': 'MEFGLSWVFLVAILKGVQCEMQLVESGGGLVQPGGSLKLSCAASGFSFSDSAMHWVRQASGKGLEWVGRIRSKPNNYATAYAASVKGRFTISRDDSKNTAYLQMNSLKTEDTAVYYCTSSPQLELYVDYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                        'L': 'MDMRVPAQLLGLLLLWLPGAKCDIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYKASRLQSGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCLQYDTYPWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '114_115': {'H': 'MKHLWFFLLLVAAPRWVLSQVQLQESGPGLVKPSQTLSLTCTVSGGSINSGGYYWSWIRQHPGKGLEWIGYIHYSGSTYYSPSLKSRITISVDTSKDQFSLKLSSVTAADTAVYYCARENDWSWFDPWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                        'L': 'MEAPAQLLFLLLLWLPDTTGEIVLTQSPATLSLSPGERATLSCRASQSVRSYLAWYQQKPGQAPRLLIYDASNRATGIPARFSGSGSGTDFTLTISSLEPEDFAVYYCQQRSNWPKTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '116_117': {'H': 'MDLLHKNMKHLWFFLLLVAAPRWVLSQVQLRQWGAGLLKPSEILSRTCAVFGGSLSGYSWSWIRQPPGKGLEWIGEITYSGNTRYNPSLKSRVTVSVDTSKNQFSLRLSSVTAADTAVYFCARVMNGVVPSPLGGLGPWYSYDAMDVWGQGTTVIVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                        'L': 'MAWTPLWLTLLTLCIGSVVSSELTQDPAVSVALGQTVRITCQGDSLRKYYASWYQQKPGQAPVLVIYGKNSRPSGIPDRFSGSTSGDTASLAIAETQAEDEAEYYCHSRDNTGDHVFGTGTKVTILGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASS'},
            '118_119': {'H': 'MKHLWFFLLLVAAPRWVLSQVQLQESGPGLLKPSETLSLSCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTSYNPSLKSRVAISVDTSKNQFSLKLSSVTAADTAVYYCATDYYDSSGYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                        'L': 'MDMRVPAQLLGLLLLWFPGSRCDIQMTQSPSSVSASVGDRVTITCRASQDIGSWSAWYQQKPGKAPKLLIYAVSNLQSGVPSRFSGSGSGTDFTLTISGLQPEDFATYYCQQANSFRTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}}

for antibody_i in list(seq_dict_batch1_cloned.keys()):
    seq_dict[antibody_i] = {}
    seq_dict[antibody_i]['H'] = seq_dict_batch1_cloned[antibody_i]['H']
    seq_dict[antibody_i]['L'] = seq_dict_batch1_cloned[antibody_i]['L']

In [4]:
#the sequences of the first batch antibodies that have not been tested (other 35 antibodies)
seq_dict_batch1_uncloned = {
            '3_4': {'H': 'MDWTWRILFLVAAATGAHSQVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPNSGGTNYAQKFQGRVTMTRDTSISTAYMELSRLRSDDTAVYYCARSLQYTYSGYDFHYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                    'L': 'MAWALLFLTLLTQGTGSWAQSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSSTLVVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'},
            '5_6': {'H': 'MEFGLSWVFLVAVIKGVQCQVQLVESGGDLVEPGGSLRLSCAASGFTFSGFFMSWIRQAPGKGLEWVSYISNGANAINYADSVKGRFIISRDDAKNSLYLQMHSLRADDTAIYFCARVGRAAWEIDYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV',
                    'L': 'MASFPLLLTLLTHCAGSWAQSVLTQPPSASGTPGQTVIISCSLTTSNIRSDSVTWYQQLPGMAPKLLIYNNNRRPSGVPDRIYGSKSGSSASLAISGLQPEDEADYYCAAWDDSLNGWVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'},
            '9_10': {'H': 'MELGLRWVFLVAILEGVQCEGRLEESGGGLVEPGGSLRLSCAASGFPFSNYTMNWVRQSPGKGLQWVSSITSGGFYTFYADSVKGRFTISRDNAKNLLFLQMNSLGAEDTAVYYCARESIADLWGHGTMVIVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
                     'L': 'MEAPAQLLFLLLLWLPDTTGEIVMTQSPATLSVSPGERATLSCRASQSVSSDLAWYQQRPGQAPSLLIYGASTRATGVPARFSGSGSGTEYTLTISGLQSEDFAVYYCLQYNTWPRTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '21_22': {'H': 'MELGLSWIFLLAILKGVQCEVQLVESGGGLVQPGRSLRLSCAASGFIFDDYAMHWVRQAPGKGLEWVSGISWNSGRIGYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTALYYCAKDTRRIAAAIMDVWGQGTTVTVSSGSASAPTLFPLVSCENSPSDTSSV',
                      'L': 'MDMRVPAQLLGLLLLWLRGARCDIQMTQSPSSLSASVGDRVTITCRASQSISSYLNWYQQKPGKAPQLLIYAASSLQSGVPSRFSGSGSGTDFTLTISNLQPEDFATYYCQQSYNTLYTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'},
            '23_24': {'H': 'MESWLSWVFLVAISKGVQCEEQVVETGGGLIQPGGSLRLSCAASGFSVSTNYMSWVRQAPGKGLEWVSVIYRGGNTHHADFVKGRFTISRDNSQNMVYLQMNSLRAEDTALYYCARRMRSAWGMDVWGQGTTVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MAWALLLLTLLTQGTGSWAQSALTQPPSASGSPGQSVTISCTGTSSDIGGYNYVSWYQQHPGKAPKLMIYEVNKRPSGVPDRFSGSKSGNTASLTVSGLQAEDEADYYCSSYAGSNNLVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '27_28': {'H': 'MKHLWFFLLLVAAPRWVLSQVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARGGSSGYYYVSSRYYFDYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MEAPAQLLFLLLLWLPDTTGEIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRLLIYDASNRATGIPARFSGSGSGTDFTLTISSLEPEDFAVYYCQQRSNWPPITFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '29_30': {'H': 'MDWTWRILFLVAAATGAHSQVQLVQSGTEVREPGASVRVSCKASGYSFTDYYIHWVRQAPGQGLEWMGWINPNSGATKYAQRFQGWVTTTRDTSIRTGYMELTNLKSDDTAIYYCARAARGGHPTGVFDMWGQGTMVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MAWALLFLTLLTQGTGSWAQSALTQPASVSGSPGQSVTISCAGSSSDVGGYSYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSITLVFGTGTKVTVLRQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASS'}, 
            '33_34': {'H': 'MDWTWRILFLVAAATGAHSQVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPNSGGTNYAQKFQGRVTMTRDTSISTAYMELSRLSSDDTAVYYCARFSPTGDFDWFGPYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MAWALLFLTLLTQGTGSWAQSALTQPASVSGSPGQSITISCTGTSSDVGAYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSGSTWVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '35_36': {'H': 'MDWTWRILFLVAAATGAHSQVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPNSGGTNYAQKFQGRVTMTRDTSISTAYMELSRLRSDDTAVYYCGLSLGELPWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MEAPAQLLFLLLLWLPDTTGEIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRLLIYDASNRATGIPARFSGSGSGTDFTLTISSLEPEDFAVYYCQQRSNWRGTFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '37_38': {'H': 'MDILCSTLLLLTVPSWVLSQVTLKESGPALVKPTQTLTLTCSFSGFSLSTSGMRVSWIRQPPGKALEWLARINWDDDKFYSTSLKTRLTISKDTSKNQVVLTMTNMDPVDTATYYCARIARGSISDAFDIWGQGTMVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MDMRVPAQLLGLLLLWLPGTRCDIQMTQSPSSLSASVGDRVTITCRASQGISNSLAWYQQKPGKAPKLLLYAASRLESGVPSRFSGSGSGTDYTLTISSLQPEDFATYSCQQYYSAPWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '41_42': {'H': 'MEFGLSWLFLVAILKGVQCEVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQGPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDYGGGNYNDYWGQGTLVTVSSAPTKAPDVFPIISGCRHPKDNSPVVLACLITGYH', 
                      'L': 'MDMRVPAQLLGLLLLWLPGAKCDIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKVLIYKASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYPITFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '45_46': {'H': 'MEFGLSWLFLVAILKGVQCEVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMTWVRQAPGKGLEWVSAISASGGNTYYADSVKGRFTISRDNSKNTMYLEMNSLGAEDTAVYYCAKEDYLPEYSSGWEFPIDYWGQGTPVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MAWTPLLFLTLLLHCTGSLSQLVLTQSPSASASLGASVKLTCTLSSGHSSYAIAWHQQQPEKGPRYLMKLNSDGSHSKGDGIPDRFSGSSSGAERYLTISSLQSEDEADYYCQTWGTGIQVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '47_48': {'H': 'MEFGLSWLFLVAILKGVQCEVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSSLSGSGGSTYYAGSVKGRFTMSRDTSKNTLYLQMNSLRAEDTAIYYCAKDRKPGIAVTGTQEDWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MDMRVPAQLLGLLLLWFPGSRCDIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKLLIYAASNLQSGVPSRFSGSGSGTDFTLTISSLEPEDCATYYCQQANSFPFTFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '49_50': {'H': 'MEFGLSWLFLVAILKGVQCEVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCASGSAIVATISDYYGMDVWGQGTTVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKLLIYDASNLETGVPSRFSGSGSGTDFTFTISSLQPEDIATYYCQQYDNLPLLFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '51_53': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVASISYDGSEYYAESVKGRFTISRDNSKSTLHLQMKSLRAEDTAVYYCAKNGGPYCSGGGCYGSYFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MRLPAQLLGLLMLWVPGSSGDVVMTQSPLSLPVTLGQPASISCRSSQSLVYSDGNTYLNWFQQRPGQSPRRLIYKVSNRDSGVPDRFSGSGSGTHFTLKISRVEAEDVWLYYCMQGTHWLFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '54_55': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDSFQWLVPQHYGMDVWGQGTTVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MRLPAQLLGLLMLWVSGSSGDIVMTQSPLSLPVTPGEPASISCRSSQSLLHSNGYNYLDWYLQKPGQSPQLLIYLGSNRASGVPDRFSGSGSGTDFTLKISRVEAEDVGVYYCMQALQTPPYTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '56_57': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSDYAMHWVRQAPGKGLEWVAVISYDGSNKYYTDSVKGRFTISRDNSKDTLYLQMNSLRVEDTAVYYCAREAVAGRSPGGYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MRLPAQLLGLLMLWVPGSNGDVVMTQSPLSLPVTFGQPASISCRSSQSLVHSDGNTYLNWFQQRPGQSPRRLLYEVSNRDSGVPDRFSGSGSGTDFTLKIRRVEAEDVGVYYCMQGTHWPLTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '58_59': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSSYTMHWVRQTPGKGLAWVAVVLYDGSNKYYADSVKGRFTVSRDNSKNTLYLQMNSLRAEDTAIYYCARGYCSGGSCYPTAYYFDNWGQGTPVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MDMRVPAQLLGLLLLWLPGAKCDIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYKASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYSSYSPLTFGPGTKVDIRRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '60_61': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGRSLRLSCAASGFTFSRYCMHWVRQAPGKGLEWVAVIWYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARDPGIAAADYYYYGMDVWGQGTTVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MAWTVLLLGLLSHCTGSVTSYVLTQPPSVSVAPGKTARITCGGNNIGSKSVHWYQQKPGQAPVLVIYYDSDRPSGIPERFSGSNSGNTATLTISRVEAGDEADYYCQVWDSSSDHVVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '62_63': {'H': 'MEFGLSWVFLVALLRGVQCQVQLVESGGGVVQPGTSLRVSCAASGFTFRSSAMHWVRQAPGKGLEWVAVISFDGNNKYYADSVKGRFTISRDNSKNTLYLEMNSLGAEDTGLYYCARPNQTDLRRGKMDVWGQGTTVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MAWALLLLSLLTQGTGSWAQSALTQPRSVSGSPGQSVTISCTGTSSDVGNYNSLSWYQQHPGEAPKLLVYDVTKRPSGVPDRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSYTWVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '64_65': {'H': 'MELGLCWVFLVAILEGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTFSSYEMNWVRQAPGKGLEWVSYISSSGSTIYYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCSRGSGWYVLDYWGQGTLVTVSSAPTKAPDVFPIISGCRHPKDNSPVVLACLITGYH', 
                      'L': 'MRLPAQLLGLLMLWVSGSSGDIVMTQSPLSLPVTPGEPASISCRSSQSLLHSNGYNYLDWYLQKPGQSPQLLIYLGSNRASGVPDRFSGSGSGTDFTLKISRVEAEDVGVYYCMQALQTPPTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '66_67': {'H': 'MELGLSWVFLAAILEGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTFSRYWMNWVRQAPGKGLEWVAKIKQDGSEKYYVDSVKGRFTISRDNAQNSLYLQMNSLRAEDTAVFYCASQYTSSSGFDYWGQGTLVTVSSASPTSPKVFPLSLCSTQPDGNVVIACLVQGFFPQEPLSVTWSESGQGVTARNF', 
                      'L': 'MAWTALLLSLLAHFTGSVASYELTQPLSVSVALGQTARITCGESNIGTKIVHWYQQKPGQAPVLVIYTDSSRPSGIPERFSGSNSGNTATLTISRAQAGDEADYYCQVWDSSTAVFGGGTKLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '68_69': {'H': 'MELGLSWVFLVAILEGVQCEVQLAESGGGLVQRGGSLRLSCSASGFTFRVYWMSWVRQAPGKGLEWVANIKQDGSEKYYVDSVKGRFTISRDNGKSSLFLQMNSLRAEDTAIYYCARVASIYYGLDVWGQGTTVTVSSAPTKAPDVFPIISGCRHPKDNSPVVLACLITGYH', 
                      'L': 'MDMRVPAQLLGLLLLWLRGARCDIQMTQSPSSLSASVGDRVTVTCRASQSIGYYLNWYQQKPGKAPKLLIYAASSLKSGVPSRFAGSGSETDFTLTISSLQPEDFATYYCQQTYRPPLFTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '70_71': {'H': 'MEFGLSWVFLVVILQGVQCEVQLVESGGGLVQPGESLRLSCAASGFTFSDHYMDWVRQAPGKGLEWVGRARNKANSHTTEYAASVKGRFTISRDDSKNLLYLQMNSLKTDDTAVYYCVRVGILTGYYLDYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MDMRVPAQLLGLLLLWFPGARCDIQMTQSPSSLSASVGDRVTITCRASQGIRSGLGWYQQKPGKAPKRLIYGASSLQSGVPSRFSGSGSGTEFTLTISSLQPEDFATYYCLQHNSYPLTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '72_73': {'H': 'MEFGLSWVFLIAILKGVQCEVQLLESGGGLVQPGGSLKLSCAASGFSFSASSIHWVRQASGKGLEWVGRIRSRINGYATPLTASVEGRFTFSRDDSSNTAYLQMNNLKTGDTAVYYCIVSWAVDYDGMDVWGQGTTVTVSSASPTSPKVFPLSLCSTQPDGNVVIACLVQGFFPQEPLSVTWSESGQGVTARNF', 
                      'L': 'MDMRVPAQLLGLLLLWFPGARCDTQMTQSPSAMSASVGDRVTITCRASQGISNSLAWFQQKPGKVPKRLIYRASSLQSGVPSRFSGSGSGTEFTLTISSLQPEDFATYYCLQHDTYPYTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '74_75': {'H': 'MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGRSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTTYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCARGYGTGSYNSYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MDMRVPAQLLGLLLLWLRGARCDIQMTQSPSSLSASVGDRVTITCRASQSISSYLNWYQQKPGRAPKLLIYAASTLQSGVPSRFSGSGSGADFTLTISSLQPEDFAIYYCQQGYSPSWTFGQGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '76_77': {'H': 'MKHLWFFLLLVAAPRWVLSQVQLQESGPGLVKPSQTLSLTCTVSGDSISSGGYYWSWIRQHPGKGLEWIGYITHSGSTYYNPSLKSRVIISVDTSKIQFSLRLSSVTAADTAVYYCARDRGRDYFDYWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MTCSPLLLTLLIHCTGSWAQSVLTQPPSVSAAPGQKVTISCSGSSSNIGNNYVSWYQQLPGTAPKLLIYENNKRPSGIPDRFSGSKSGTSATLGITGLQTGDEADYYCGTYDSSLSQAVFGGGTQLTVLGQPKAAPSVTLFPPSSEELQANKATLVCLVSDFYPGAVTVAWKADGSPVKVGVETTKPSKQSNNKYAASS'}, 
            '78_79': {'H': 'MDLLHKNMKHLWFFLLLVAAPRWVLSQVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCAGRPGYYDFWSGYYSHWGQGTLVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                      'L': 'MRLPAQLLGLLMLWVSGSSGDIVMTQSPLSLPVTPGEPASISCRSSQSLLHSNGYNYLDWYLQKPGQSPQLLIYLGSNRASGVPDRFSGSGSGTDFTLKISRVEAEDVGVYYCMQALQTPHTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '80_81': {'H': 'MDLLHKNMKHLWFFLLLVAAPRWVLSQVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARARRRGQQLVGGYYYYYGMDVWGQGTTVTVSSAPTKAPDVFPIISGCRHPKDNSPVVLACLITGYH', 
                      'L': 'MDMRVPAQLLGLLLLWFPGSRCDIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQANSFPFTFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '82_83': {'H': 'MKHLWFFLLLVAAPTWVLSQVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGRGLEWIGYIYYSGSTNYNPSLKSRVTLSVDTSNNHFSLKLGSVTAADTAVYYCARHGDFWSGKSWFDPWGQGTLVTVSSAPTKAPDVFPIISGCRHPKDNSPVVLACLITGYH', 
                      'L': 'MAGFPLLLTLLTHCAGSWAQSVLTQPPSASGTPGQRVTISCSGISSNIGSNYVYWYQHLPGTAPKLLIYRDNLRPSGVPDRFSGSKSGTSVSLAISGLRSEDEADYYCAAWDDSLSGWVFGGGTKVTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASS'}, 
            '84_85': {'H': 'MKHLWFFLLLVAAPRWVLSQLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARLFRLWPPHWFDPWGQGTLVTVSSAPTKAPDVFPIISGCRHPKDNSPVVLACLITGYH', 
                      'L': 'METPAQLLFLLLLWLPDTTGEIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGSSRWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '90_91': {'H': 'MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSLIYSGGSTYYADSVKGRFTISRDYSKNTLYLQMNSLRAEDTATYYCARERPRGAGEYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG', 
                      'L': 'MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQASQDINIYLNWYQQKPGKAPKLLIYDASNLETGVPSRFSGSGSGTDFTFTISSLQPEDIATYYCHQHDNLPRTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '98_99': {'H': 'MDWIWRILFLVGAATGAHSQMQLVQSGPEVKKPGTSVKVSCKASGFTFSSSAMQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTNTAYMELSSLRSEDTAVYYCAAPHCNRTNCFDGFDIWGQGTMVTVSLASPTSPKVFPLSLCSTQPDGNVVIACLVQGFFPQEPLSVTWSESGQGVTARNF', 
                      'L': 'METPAQLLFLLLLWLPDTTGEIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYVASSRATGIPDRFSGSGSGTDFTLTISRLEPEDLAVYYCQQYGSSPWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}, 
            '102_103': {'H': 'MDILCSTLLLLTVPSWVFSQVTLRESGPALVKPTQTLTLTCTFSGFSLTTSGMCVTWVRQSPGKALEWLARIDWDDDRYYNISLKTRLTISKDISKNQVVLTMTDMDPVDTATYYCARIRDYVWGTTGHFDYWGPGMLVTVSSASPTSPKVFPLSLCSTQPDGNVVIACLVQGFFPQEPLSVTWSESGQGVTARNF', 
                        'L': 'MASFPLLLTLLTHCAGSWAQSLLTQPPSASGTPGQTITIPCSGSSSNIGSNTVNWYQQLPGAAPKLLIYNNDQRPSGVPERFSGSKSGTSASLAISGLQSEDEADYYCAAWHDSLNSDVFGTGTKVTVLRQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASS'}, 
            '112_113': {'H': 'MKHLWFFLLLVAAPRWVLSQVQLQESGPGLVKPSQTLSLTCTVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCASFVRFLEWLLPDDDAFDIWGQGTMVTVSSGSASAPTLFPLVSCENSPSDTSSV', 
                        'L': 'MVLQTQVFISLLLWISGAYGDIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDN'}}

for antibody_i in list(seq_dict_batch1_uncloned.keys()):
    seq_dict[antibody_i] = {}
    seq_dict[antibody_i]['H'] = seq_dict_batch1_uncloned[antibody_i]['H']
    seq_dict[antibody_i]['L'] = seq_dict_batch1_uncloned[antibody_i]['L']

In [9]:
#14 clinical antibodies
seq_dict_clinical = {
    'BAM': {'H': 'QVQLVQSGAEVKKPGSSVKVSCKASGGTFSNYAISWVRQAPGQGLEWMGRIIPILGIANYAQKFQGRVTITADKSTSTAYMELSSLRSEDTAVYYCARGYYEARHYYYYYAMDVWGQGTAVTVSS', 
            'L': 'DIQMTQSPSSLSASVGDRVTITCRASQSISSYLSWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTITSLQPEDFATYYCQQSYSTPRTFGQGTKVEIK'}, 
    'ETE': {'H': 'EVQLVESGGGLVQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSVIYSGGSTFYADSVKGRFTISRDNSMNTLFLQMNSLRAEDTAVYYCARVLPMYGDYLDYWGQGTLVTVSS', 
            'L': 'DIVMTQSPSSLSASVGDRVTITCRASQSISRYLNWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYSTPPEYTFGQGTKLEIK'},
    'CAS': {'H': 'QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYITYSGSTIYYADSVKGRFTISRDNAKSSLYLQMNSLRAEDTAVYYCARDRGTTMVPFDYWGQGTLVTVSS', 
            'L': 'DIQMTQSPSSLSASVGDRVTITCQASQDITNYLNWYQQKPGKAPKLLIYAASNLETGVPSRFSGSGSGTDFTFTISGLQPEDIATYYCQQYDNLPLTFGGGTKVEIK'}, 
    'IMD': {'H': 'QVQLVESGGGVVQPGRSLRLSCAASGFTFSNYAMYWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRTEDTAVYYCASGSDYGDYLLVYWGQGTLVTVSS', 
            'L': 'QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSKRPSGVSNRFSGSKSGNTASLTISGLQSEDEADYYCNSLTSISTWVFGGGTKLTVL'}, 
    'CIL': {'H': 'EVQLVESGGGLVKPGGSLRLSCAASGFTFRDVWMSWVRQAPGKGLEWVGRIKSKIDGGTTDYAAPVKGRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTTAGSYYYDTVGPGLPEGKFDYWGQGTLVTVSS', 
            'L': 'DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLMYWASTRESGVPDRFSGSGSGAEFTLTISSLQAEDVAIYYCQQYYSTLTFGGGTKVEIK'}, 
    'TIX': {'H': 'QMQLVQSGPEVKKPGTSVKVSCKASGFTFMSSAVQWVRQARGQRLEWIGWIVIGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSSISCNDGFDIWGQGTMVTVSS', 
            'L': 'EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQHYGSSRGWTFGQGTKVEIK'}, 
    'SOT': {'H': 'QVQLVQSGAEVKKPGASVKVSCKASGYPFTSYGISWVRQAPGQGLEWMGWISTYNGNTNYAQKFQGRVTMTTDTSTTTGYMELRRLRSDDTAVYYCARDYTRGAWFGESLIGGFDNWGQGTLVTVSS', 
            'L': 'EIVLTQSPGTLSLSPGERATLSCRASQTVSSTSLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQHDTSLTFGGGTKVEIK'}, 
    'BEB': {'H': 'QITLKESGPTLVKPTQTLTLTCTFSGFSLSISGVGVGWLRQPPGKALEWLALIYWDDDKRYSPSLKSRLTISKDTSKNQVVLKMTNIDPVDTATYYCAHHSISTIFDHWGQGTLVTVSS', 
            'L': 'QSALTQPASVSGSPGQSITISCTATSSDVGDYNYVSWYQQHPGKAPKLMIFEVSDRPSGISNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTTSSAVFGGGTKLTVL'}, 
    'REG': {'H': 'QITLKESGPTLVKPTQTLTLTCSFSGFSLSTSGVGVGWIRQPPGKALEWLALIDWDDNKYHTTSLKTRLTISKDTSKNQVVLTMTNMDPVDTATYYCARIPGFLRYRNRYYYYGMDVWGQGTTVTVSS', 
            'L': 'ELVLTQPPSVSAAPGQKVTISCSGSSSNIGNNYVSWYQQLPGTAPKLLIYDNNKRPSGIPDRFSGSKSGTSATLGITGLQTGDEADYYCGTWDSSLSAGVFGGGTELTVL'}, 
    'AMU': {'H': 'EVQLVESGGGLVQPGGSLRLSCAASGITVSSNYMNWVRQAPGKGLEWVSLIYSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYHCARDLVVYGMDVWGQGTTVTVSS', 
            'L': 'EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGSSPTFGQGTKLEIK'}, 
    'ROM': {'H': 'QVQLVQSGSELKKPGASVKVSCKASGYTFTTYVMNWVRQAPGQGLEWMGWINTNTGNPTYAQGFTGRFVFSLDTSVSTASLQISSLKAEDTAVYYCSSEITTLGGMDVWGQGTTVTVSS', 
            'L': 'SYVLTQPPSVSVAPGKTARITCGGNNIGSKSVHWYQQKPGQAPVLVIYYDSDRPSGIPERFSGSNSGNTATLTISGVEAGDEADYYCQVWDSISDHRVFGGGTKLTVL'}, 
    'ADI': {'H': 'EVQLVESGGGLVKPGGSLRLSCAASGFTFSSYYMNWVRQAPGKGLEWVSSISEDGYSTYYPDSLKGRFTISRDSAKNSLYLQMNSLRADDTAVYYCARDFSGHTAWAGTGFEYWGQGTLVTVSS', 
            'L': 'QSVLTQPPSVSGAPGQRITISCTGSSSNIGAGYDVHWYQQLPGTAPKLLIYGSSSRNSGVPDRFSGSKSGTSASLAITGLQAEDEADYYCQSYDSSLSVLYTFGTGTKVTVL'},
    'C135': {'H': 'QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYAMHWVRQAPGKGLEWVAVIPFDGRNKYYADSVTGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCASSSGYLFHSDYWGQGTLVTVSS', 
             'L': 'DIQMTQSPSTLSASVGDRVTITCRASQSISNWLAWFQQKPGKAPKLLIYEASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYPWTFGQGTKVEIK'}, 
    'C144': {'H': 'EVQLVESGGGLIQPGGSLRLSCAASGFTVSNNYMSWVRQAPGKGLEWVSVIYSGGSTYYADSVKGRFTISRDKSKNTLYLQMNRLRAEDTAVYYCAREGEVEGYNDFWSGYSRDRYYFDYWGQGTLVTVSS', 
             'L': 'QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSSTRVFGTGTKVTVL'}}


In [1]:
#calculate the passive residues
p = PDBParser()
structure = p.get_structure('6w41', '6w41_c.pdb')
model = structure[0]
dssp = DSSP(model, '6w41_c.pdb')

passive_residue = []
for resi_i in list(dssp.keys()):
    if(dssp[resi_i][3] >= 0.4):
        #print(dssp[resi_i])
        passive_residue.append(resi_i[1][1])

In [7]:
#seq_dict contains all the original sequences provided

{'H': 'MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGGSLRLSCVASGITVSANYMSWVRQAPGKGLEWVSVIYSGGSTFYADSVKGRFTISRDNSKNTLYLQMNNLRADDTAVYSCARDFRGATAFDIWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSG',
 'L': 'MAWALLFLTLLTQGTGSWAQSALTQPASVSGSPGQSITISCTATSSDVDDYNYVSWYQQHPGKAPKLLIYDVNNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSSTGVFGSGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASS'}

In [14]:
#Preprocessing: chopping for clustering.

patient_antibody_list = list(seq_dict)
clinical_antibody_list = list(seq_dict_clinical)

antibody_list_all_with_clinical = patient_antibody_list + clinical_antibody_list

input = open('heavy chain alignment result.txt').readlines()
heavy_chain_align_result_dict = {}
for antibody_i in antibody_list_all_with_clinical:
    heavy_chain_align_result_dict[antibody_i] = ''

for row in input:
    if(len(row.split()) == 3):
        if(row.split()[0] in antibody_list_all_with_clinical):
            heavy_chain_align_result_dict[row.split()[0]] = heavy_chain_align_result_dict[row.split()[0]] + row.split()[1]

count = np.zeros(len(heavy_chain_align_result_dict[antibody_i]))
for antibody_i in clinical_antibody_list:
    for i in range(len(count)):
        if(heavy_chain_align_result_dict[antibody_i][i] != '-'):
            count[i] += 1

for antibody_i in antibody_list_all_with_clinical:
    heavy_chain_align_result_dict[antibody_i] = heavy_chain_align_result_dict[antibody_i][56:216].replace('-', '')

input = open('light chain alignment result.txt').readlines()
light_chain_align_result_dict = {}
for antibody_i in antibody_list_all_with_clinical:
    light_chain_align_result_dict[antibody_i] = ''

for row in input:
    if(len(row.split()) == 3):
        if(row.split()[0] in antibody_list_all_with_clinical):
            light_chain_align_result_dict[row.split()[0]] = light_chain_align_result_dict[row.split()[0]] + row.split()[1]

count = np.zeros(len(light_chain_align_result_dict[antibody_i]))
for antibody_i in clinical_antibody_list:
    for i in range(len(count)):
        if(light_chain_align_result_dict[antibody_i][i] != '-'):
            count[i] += 1

for antibody_i in antibody_list_all_with_clinical:
    light_chain_align_result_dict[antibody_i] = light_chain_align_result_dict[antibody_i][54:179].replace('-', '')

In [19]:
seq_dict_all = {}
for antibody_i in heavy_chain_align_result_dict:
    seq_dict_all[antibody_i] = {}
    seq_dict_all[antibody_i]['H'] = heavy_chain_align_result_dict[antibody_i]
    seq_dict_all[antibody_i]['L'] = light_chain_align_result_dict[antibody_i]    

#some antibodies are removed later, as failed to get the docking structure.

In [ ]:
#Determine ACE2 binding residues on RBD

In [27]:
pymol.cmd.load('6lzg.pdb')
pymol.cmd.remove('resn hoh')
# and not name H*
pymol.cmd.select('selected_rot', '(chain B) within 5 of (chain A and not name H*)')
pymol.cmd.save('6lzg_ace2binding.pdb', 'selected_rot')
pymol.cmd.delete('all')
pymol.cmd.load('6m0j.pdb')
pymol.cmd.remove('resn hoh')
pymol.cmd.select('selected_rot', '(chain E) within 5 of (chain A and not name H*)')
pymol.cmd.save('6m0j_ace2binding.pdb', 'selected_rot')
pymol.cmd.delete('all')
pymol.cmd.load('6m17.pdb')
pymol.cmd.remove('resn hoh')
pymol.cmd.select('selected_rot', '(chain F) within 5 of (chain D and not name H*)')
pymol.cmd.save('6m17_ace2binding_fd.pdb', 'selected_rot')
pymol.cmd.delete('all')
pymol.cmd.load('6m17.pdb')
pymol.cmd.remove('resn hoh')
pymol.cmd.select('selected_rot', '(chain E) within 5 of (chain B and not name H*)')
pymol.cmd.save('6m17_ace2binding_eb.pdb', 'selected_rot')
pymol.cmd.delete('all')

resi_list = []
for file_i in ['6lzg_ace2binding.pdb', '6m0j_ace2binding.pdb', '6m17_ace2binding_eb.pdb', '6m17_ace2binding_fd.pdb']:
    input = open(file_i).readlines()
    for row in input:
        row_split = row.split()
        if(len(row_split) > 5):
            if(row_split[0] == 'ATOM'):
                resi_list.append(row[23:26])
    print(len(np.unique(np.array(resi_list))))
    os.remove(file_i)
    
ace2_binding_residue = np.unique(np.array(resi_list, dtype = int))
print(len(ace2_binding_residue))
print(ace2_binding_residue)
#select resi 417+446+447+449+453+455+456+473+475+476+477+484+486+487+489+490+493+494+495+496+498+500+501+502+503+505

 PyMOL not running, entering library mode (experimental)
23
23
26
26
26
[417 446 447 449 453 455 456 473 475 476 477 484 486 487 489 490 493 494
 495 496 498 500 501 502 503 505]


In [ ]:
#ABodybuilder
number_to_character = {'0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'}

for pair_i in list(seq_dict.keys()):
    heavy_chain_name = ''
    light_chain_name = ''
    for name_i in range(len(pair_i.split('_')[0])):
        heavy_chain_name = heavy_chain_name + number_to_character[pair_i.split('_')[0][name_i]]
    for name_i in range(len(pair_i.split('_')[1])):
        light_chain_name = light_chain_name + number_to_character[pair_i.split('_')[1][name_i]]
    if(not (os.path.exists('wt/ABodyBuilder_results/' + pair_i + '.pdb')) | ((heavy_chain_name + '_' + light_chain_name) in ['onefoureighteight_onefoureightnine', 'oneeightnineseven_oneeightnineeight', 'twosevenfourzero_twoseventhreenine', 'threezeroeightfive_threezeroeightfour'])):
        print('name: ' + heavy_chain_name + '_' + light_chain_name)
        #You may need to download the chrome driver and make sure the version is consistent with the module used.
        abodybuilder = webdriver.Chrome('C:/Users/wuwei/chromedriver.exe')
        abodybuilder.get('http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/abodybuilder/')
        abodybuilder.find_element_by_name('hchain').send_keys(seq_dict[pair_i]['H'])
        abodybuilder.find_element_by_name('lchain').send_keys(seq_dict[pair_i]['L'])
        abodybuilder.find_element_by_name('jobname').clear()
        abodybuilder.find_element_by_name('jobname').send_keys(heavy_chain_name + '_' + light_chain_name)
        abodybuilder.find_element_by_class_name('btn').click()
        wait = WebDriverWait(abodybuilder, 500)
        wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'table-results')))
        abodybuilder.find_elements_by_xpath("//*[@class='bluetoorangelink']")[3].click()
        time.sleep(30)
        abodybuilder.quit()

In [ ]:
#proabce2
if(os.path.isfile('C:/Users/wuwei/Downloads/heavy-pred.csv')):
    os.remove('C:/Users/wuwei/Downloads/heavy-pred.csv')
if(os.path.isfile('C:/Users/wuwei/Downloads/light-pred.csv')):
    os.remove('C:/Users/wuwei/Downloads/light-pred.csv')
path = 'wt/ABodyBuilder_results_processed/'
pdb_list = glob.glob(path + '*_H.fasta')
for pdb_i in range(len(pdb_list)):
    pdb_name = pdb_list[pdb_i].split('\\')[1].split('.fasta')[0]
    pdb_name = pdb_name[0:(len(pdb_name) - 2)]
    if((not pdb_i in [384, 870]) & (not os.path.isfile('wt/proabc2_results/' + pdb_name + '_heavy.csv'))):
        print(pdb_name)
        h_chain = open(path + pdb_name + '_H.fasta').readlines()
        h_seq = ''
        for i in np.arange(1, len(h_chain)):
            h_seq = h_seq + h_chain[i].split('\n')[0]
        l_chain = open(path + pdb_name + '_L.fasta').readlines()
        l_seq = ''
        for i in np.arange(1, len(l_chain)):
            l_seq = l_seq + l_chain[i].split('\n')[0]
        proabc2 = webdriver.Chrome('C:/Users/wuwei/chromedriver.exe')
        proabc2.get('https://wenmr.science.uu.nl/proabc2/')
        proabc2.find_element_by_class_name('cc-dismiss').click()
        wait = WebDriverWait(proabc2, 300)
        proabc2.find_element_by_id('sequence_heavy').send_keys('>heavy\n' + h_seq)
        proabc2.find_element_by_id('sequence_light').send_keys('>light\n' + l_seq)
        wait.until(EC.element_to_be_clickable((By.ID, 'submit')))
        button = proabc2.find_element_by_id('submit')
        proabc2.execute_script("arguments[0].click();", button)
        wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'heavy-pred.csv')))
        button = proabc2.find_element_by_link_text('heavy-pred.csv')
        proabc2.execute_script("arguments[0].click();", button)
        current_link = proabc2.current_url
        time.sleep(6)
        proabc2.quit()
        proabc2 = webdriver.Chrome('C:/Users/wuwei/chromedriver.exe')
        proabc2.get(current_link)
        button = proabc2.find_element_by_link_text('light-pred.csv')
        proabc2.execute_script("arguments[0].click();", button)
        time.sleep(6)
        proabc2.quit()
        move('C:/Users/wuwei/Downloads/heavy-pred.csv', 'wt/proabc2_results/' + pdb_name + '_heavy.csv')
        move('C:/Users/wuwei/Downloads/light-pred.csv', 'wt/proabc2_results/' + pdb_name + '_light.csv')


In [ ]:
#Haddock
#You may need to register an account
username = ''
password = ''

pair_i = 0
pair_i_start = pair_i
abodybuilder_input_path = 'wt/ABodyBuilder_results_processed/'
proabc2_input_path = 'wt/proabc2_results/'
pdb_list_proabc2 = glob.glob(proabc2_input_path + '*_heavy.csv')
haddock_output_path = 'wt/haddock_results/'
pair_list = []
for pdb_i in pdb_list_proabc2:
    pair_list.append(pdb_i.split('\\')[1].split('_heavy.csv')[0])
for pair_i in range(len(pair_list)):
    if(pair_list[pair_i] in ['1281_1282', '9460_9461', '2854_2855', '2051_2052']):
        proabc2_result_h = open(proabc2_input_path + pair_list[pair_i] + '_heavy.csv').readlines()
        proabc2_result_l = open(proabc2_input_path + pair_list[pair_i] + '_light.csv').readlines()
        serial = ''
        h_chain_length = len(proabc2_result_h) - 1
        for resi_i in np.arange(1, len(proabc2_result_h)):
            input_temp = proabc2_result_h[resi_i].split(',')
            if((float(input_temp[3]) > 0.4) and (float(input_temp[4]) > 0.3) and (float(input_temp[5].split('\n')[0]) > 0.3)):
                if(len(serial) > 0):
                    serial = serial + ', ' + str(resi_i)
                else:
                    serial = str(resi_i)
        for resi_i in np.arange(1, len(proabc2_result_l)):
            input_temp = proabc2_result_l[resi_i].split(',')
            if((float(input_temp[3]) > 0.4) and (float(input_temp[4]) > 0.3) and (float(input_temp[5].split('\n')[0]) > 0.3)):
                if(len(serial) > 0):
                    serial = serial + ', ' + str(resi_i + len(proabc2_result_h) - 1)
                else:
                    serial = str(resi_i)    
        haddock = webdriver.Chrome('C:/Users/wuwei/chromedriver.exe')
        wait = WebDriverWait(haddock, 30)
        haddock.get('https://bianca.science.uu.nl/haddock2.4')
        haddock.find_element_by_class_name('cc-dismiss').click()
        button = haddock.find_elements_by_class_name('dropdown-toggle.nav-link.active')[0]
        haddock.execute_script("arguments[0].click();", button)
        haddock.find_element_by_id('email_dropdown').send_keys(username)
        haddock.find_element_by_id('pw_dropdown').send_keys(password)
        haddock.find_element_by_id('submit_dropdown').click()
        haddock.find_elements_by_class_name('btn')[1].click()
        haddock.find_element_by_id('runname').send_keys(pair_list[pair_i] + '_replicate')
        haddock.find_element_by_name('p1_pdb_file').send_keys(os.getcwd() + '\\' + abodybuilder_input_path + pair_list[pair_i] + '_final_cleaned.pdb')
        haddock.find_element_by_name('p2_pdb_file').send_keys(os.getcwd() + '\\' + '6w41_c.pdb')
        haddock.find_element_by_id('submit').click()
        button = haddock.find_elements_by_class_name('switch.badge-primary')[4]
        haddock.execute_script("arguments[0].click();", button)
        haddock.find_element_by_id('p1_r_activereslist_1').send_keys(serial)
        haddock.find_element_by_id('p2_r_passivereslist_1').send_keys('333, 334, 335, 339, 340, 343, 345, 346, 354, 357, 360, 367, 370, 372, 373, 375, 378, 380, 381, 385, 386, 389, 390, 405, 408, 413, 415, 417, 427, 428, 430, 440, 441, 444, 445, 446, 449, 450, 458, 459, 460, 462, 463, 466, 468, 470, 471, 474, 475, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 489, 490, 493, 499, 500, 502, 503, 505, 516, 517, 518, 519, 520, 521, 522, 527')
        haddock.find_element_by_id('submit').click()
        haddock.find_element_by_class_name('form-check-inline.d-flex').click()
        haddock.find_element_by_id('submit').click()
        time.sleep(5)
        #confirm the submission is related to the covid
        haddock.find_element_by_id('covidConfirmYes').click()
        time.sleep(5)
        haddock.quit()
        print(pair_i)

In [ ]:
#downlaod the haddock results
input = open('wt_docking_list0.txt').readlines()
for i in range(len(input)):
    if(('http' in input[i]) & ('replicate' in input[i])):
        url = 'https://wenmr.science.uu.nl/' + str(input[i].split('Go to ')[1].split('  to')[0].split('wenmr.science.uu.nl/')[1].split('_replicate')[0]) + '_replicate_summary.tgz'
        file_name = url.split('/')[-1]
        antibody_name = file_name.split('-')[1].split('_replicate')[0]
        if(not os.path.exists('wt/haddock_results/' + antibody_name)):  
            print(antibody_name)
            haddock_result = webdriver.Chrome('C:/Users/wuwei/chromedriver.exe')
            haddock_result.get(url)
            time.sleep(15)
            haddock_result.quit()
            move('C:/Users/wuwei/Downloads/' + file_name, 'wt/haddock_results/' + antibody_name + '.tgz')

In [ ]:
input_path = 'wt/haddock_results_processed/'
output_path = 'wt/haddock_results_processed_tobal/'
pair_list = os.listdir(input_path)
for pair_i in pair_list:
    pdb_list = os.listdir(input_path + pair_i)
    for pdb_i in pdb_list:
        if('_1.pdb' in pdb_i):
            shutil.copy(input_path + pair_i + '/' + pdb_i, output_path + pair_i + '_' + pdb_i.split('cluster')[1].split('_')[0] + '.pdb')

In [ ]:
input_path = 'wt/haddock_results_balrefined/'
output_path = 'wt/haddock_results_balrefined_processed/'
conformation_list = os.listdir(input_path)
for conformation_i in conformation_list:
    if(not os.path.isfile(output_path + conformation_i + '.pdb')):
        pymol.cmd.load(input_path + conformation_i + '/end/receptor.pdb')
        pymol.cmd.select('heavy_chain', 'chain A')
        pymol.cmd.alter('heavy_chain', "chain='H'")
        pymol.cmd.save(input_path + conformation_i + '/end/receptor_processed_H.pdb', 'heavy_chain')
        os.system('pdb_reres -1 ' + input_path + conformation_i + '/end/receptor_processed_H.pdb > ' + input_path + conformation_i + '/end/receptor_processed_H_reres.pdb')
        pymol.cmd.select('light_chain', 'chain B')
        pymol.cmd.alter('light_chain', "chain='L'")
        pymol.cmd.save(input_path + conformation_i + '/end/receptor_processed_L.pdb', 'light_chain')
        os.system('pdb_reres -1 ' + input_path + conformation_i + '/end/receptor_processed_L.pdb > ' + input_path + conformation_i + '/end/receptor_processed_L_reres.pdb')
        pymol.cmd.delete('all')
        pymol.cmd.load(input_path + conformation_i + '/end/ligand.pdb')
        pymol.cmd.select('rbd_chain', 'chain C')
        pymol.cmd.alter('rbd_chain', "chain='B'")
        pymol.cmd.save(input_path + conformation_i + '/end/ligand_processed.pdb', 'all')
        os.system('pdb_reres -333 ' + input_path + conformation_i + '/end/ligand_processed.pdb > ' + input_path + conformation_i + '/end/ligand_processed_reres.pdb')
        pymol.cmd.delete('all')
        pymol.cmd.load(input_path + conformation_i + '/end/receptor_processed_H_reres.pdb')
        pymol.cmd.load(input_path + conformation_i + '/end/receptor_processed_L_reres.pdb')
        pymol.cmd.load(input_path + conformation_i + '/end/ligand_processed_reres.pdb')
        pymol.cmd.save(output_path + conformation_i + '.pdb', 'all')
        pymol.cmd.delete('all')
        os.remove(input_path + conformation_i + '/end/receptor_processed_H.pdb')
        os.remove(input_path + conformation_i + '/end/receptor_processed_L.pdb')
        os.remove(input_path + conformation_i + '/end/ligand_processed.pdb')
        os.remove(input_path + conformation_i + '/end/receptor_processed_H_reres.pdb')
        os.remove(input_path + conformation_i + '/end/receptor_processed_L_reres.pdb')
        os.remove(input_path + conformation_i + '/end/ligand_processed_reres.pdb')